# Topic 1 Sequential Data Modeling with PyTorch


## RNN/LSTM/GRU and Input Parameters

In [ ]:
import torch
import torch.nn as nn

number_layers = 1 #similar to hidden layers
batch_size = 5
input_size = 10 # similar to  embedding vector size to represent a word 
hidden_size = 20 # similar to hidden vector
sequence_length = 3 # eg 3 words in a sequence or 3 days stock price data 

rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=number_layers, batch_first=True)
input = torch.randn(batch_size, sequence_length, input_size)
h0 = torch.randn(number_layers, batch_size, hidden_size)
_, hn = rnn(input, h0)

In [ ]:
input.shape, hn.shape

In [ ]:
import torch
import torch.nn as nn

number_layers = 1 #similar to hidden layers
batch_size = 5
input_size = 10 # similar to  embedding vector size to represent a word 
hidden_size = 20 # similar to hidden vector
sequence_length = 3 # # eg 3 words in a sequence or 3 days stock price dat

rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=number_layers, batch_first=False)
input = torch.randn(sequence_length, batch_size, input_size)
h0 = torch.randn(number_layers, batch_size, hidden_size)
_, hn = rnn(input, h0)

In [ ]:
input.shape, hn.shape

In [ ]:
import torch
import torch.nn as nn

number_layers = 1 #similar to hidden layers
batch_size = 5
input_size = 10 # similar to  embedding vector size to represent a word 
hidden_size = 20 # similar to hidden vector
sequence_length = 3 # eg 3 days stock price data 

rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=number_layers, batch_first=True)
input = torch.randn(batch_size, sequence_length, input_size)
h0 = torch.randn(number_layers, batch_size, hidden_size)
c0 = torch.randn(number_layers, batch_size, hidden_size)
_,(hn, cn) = rnn(input, (h0, c0))


In [ ]:
input.shape, hn.shape, cn.shape

In [ ]:
import torch
import torch.nn as nn

number_layers = 1 #similar to hidden layers
batch_size = 5 
input_size = 10 # similar to  embedding vector size to represent a word 
hidden_size = 20 # similar to hidden vector
sequence_length = 3 # eg 3 days stock price data 

rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=number_layers, batch_first=False)
input = torch.randn(sequence_length, batch_size, input_size)
h0 = torch.randn(number_layers, batch_size, hidden_size)
c0 = torch.randn(number_layers, batch_size, hidden_size)
_,(hn, cn) = rnn(input, (h0, c0))


In [ ]:
input.shape, hn.shape, cn.shape

## Time Series Forecasting (Airplane Passengers)

### LSTM Model

#### Step 1: Preprocess Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

In [ ]:
training_set = pd.read_csv('airline-passengers.csv')
training_set.head(10)

In [ ]:
training_set.describe()

In [ ]:
training_set = training_set.iloc[:,1:2].values

plt.plot(training_set, label = 'Data')
plt.show()

In [ ]:
def sliding_window(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

In [ ]:
sc = MinMaxScaler()
training_data = sc.fit_transform(training_set)

seq_length = 4
x, y = sliding_window(training_data, seq_length)

In [ ]:
train_size = int(len(y) * 0.67)
test_size = int(len(y)) - train_size

dataX = torch.Tensor(np.array(x))
dataY = torch.Tensor(np.array(y))

trainX = torch.Tensor(np.array(x[0:train_size]))
trainY = torch.Tensor(np.array(y[0:train_size]))

testX = torch.Tensor(np.array(x[train_size:len(x)]))
testY = torch.Tensor(np.array(y[train_size:len(y)]))

In [ ]:
dataX.shape, trainX.shape, testX.shape

#### Step 2: Define the Model

In [ ]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)       
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # Propagate input through LSTM
        _, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size)
        out = self.fc(h_out)
        
        return out

In [ ]:
input_size = 1
hidden_size = 2
num_layers = 1
num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

#### Step 3: Define Loss Function and Optimizer

In [ ]:
learning_rate = 0.01

criterion = torch.nn.MSELoss()    
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)


#### Step 4: Train the Model

In [ ]:
num_epochs = 2000

for epoch in range(num_epochs):
    outputs = lstm(trainX)
    
    optimizer.zero_grad()
    loss = criterion(outputs, trainY)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

#### Step 5: Evaluate the Model

In [ ]:
train_predict = lstm(dataX)

data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

data_predict = sc.inverse_transform(data_predict)
dataY_plot = sc.inverse_transform(dataY_plot)

plt.axvline(x=train_size, c='g', linestyle='--')

plt.plot(dataY_plot,'b',label='actual')
plt.plot(data_predict,'r',label='prediction')
plt.suptitle('Time Series Prediction')
plt.legend()
plt.show()

### GRU Model

#### Step 1: Preprocess the data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

In [ ]:
training_set = pd.read_csv('airline-passengers.csv')
training_set.head(10)

In [ ]:
training_set = training_set.iloc[:,1:2].values

plt.plot(training_set, label = 'Data')
plt.show()

In [ ]:
def sliding_window(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

In [ ]:
sc = MinMaxScaler()
training_data = sc.fit_transform(training_set)

seq_length = 4
x, y = sliding_window(training_data, seq_length)

In [ ]:
train_size = int(len(y) * 0.67)
test_size = int(len(y)) - train_size

dataX = torch.Tensor(np.array(x))
dataY = torch.Tensor(np.array(y))

trainX = torch.Tensor(np.array(x[0:train_size]))
trainY = torch.Tensor(np.array(y[0:train_size]))

testX = torch.Tensor(np.array(x[train_size:len(x)]))
testY = torch.Tensor(np.array(y[train_size:len(y)]))

#### Step 2: Define the Model

In [ ]:
class GRU(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(GRU, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # Propagate input through LSTM
        _, h_out = self.gru(x, h_0)
        h_out = h_out.view(-1, self.hidden_size)
        out = self.fc(h_out)
        
        return out

In [ ]:
input_size = 1
hidden_size = 2
num_layers = 1
num_classes = 1

gru = GRU(num_classes, input_size, hidden_size, num_layers)

#### Step 3: Define Loss Function and Optimizer

In [ ]:
learning_rate = 0.01

criterion = torch.nn.MSELoss()    
optimizer = torch.optim.Adam(gru.parameters(), lr=learning_rate)

#### Step 4: Train the Model

In [ ]:
num_epochs = 2000

for epoch in range(num_epochs):
    outputs = gru(trainX)
    
    optimizer.zero_grad()
    loss = criterion(outputs, trainY)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

#### Step 5: Evaluate the Model

In [ ]:
train_predict = gru(dataX)

data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

data_predict = sc.inverse_transform(data_predict)
dataY_plot = sc.inverse_transform(dataY_plot)

plt.axvline(x=train_size, c='g', linestyle='--')

plt.plot(dataY_plot,'b',label='actual')
plt.plot(data_predict,'r',label='prediction')
plt.suptitle('Time Series Prediction')
plt.legend()
plt.show()

###  RNN Model

#### Step 1: Preprocess Data (Air Passengers Dataset)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

In [ ]:
training_set = pd.read_csv('airline-passengers.csv')
training_set.head(10)

In [ ]:
training_set = training_set.iloc[:,1:2].values

plt.plot(training_set, label = 'Data')
plt.show()

In [ ]:
def sliding_window(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

In [ ]:
sc = MinMaxScaler()
training_data = sc.fit_transform(training_set)

seq_length = 4
x, y = sliding_window(training_data, seq_length)

In [ ]:
train_size = int(len(y) * 0.67)
test_size = int(len(y)) - train_size

dataX = torch.Tensor(np.array(x))
dataY = torch.Tensor(np.array(y))

trainX = torch.Tensor(np.array(x[0:train_size]))
trainY = torch.Tensor(np.array(y[0:train_size]))

testX = torch.Tensor(np.array(x[train_size:len(x)]))
testY = torch.Tensor(np.array(y[train_size:len(y)]))

#### Step 2: Define the Model

In [ ]:
class RNN(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(RNN, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # Propagate input through LSTM
        _, h_out = self.rnn(x, h_0)
        h_out = h_out.view(-1, self.hidden_size)
        out = self.fc(h_out)
        
        return out

In [ ]:
input_size = 1
hidden_size = 2
num_layers = 1
num_classes = 1

rnn = RNN(num_classes, input_size, hidden_size, num_layers)

#### Step 3: Define Loss Functoin and Optimzer

In [ ]:
learning_rate = 0.01

criterion = torch.nn.MSELoss()    
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

#### Step 4: Train the Model

In [ ]:
num_epochs = 2000

for epoch in range(num_epochs):
    outputs = rnn(trainX)
    
    optimizer.zero_grad()
    loss = criterion(outputs, trainY)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

#### Step 5: Evaluate the Model

In [ ]:
train_predict = rnn(dataX)

data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

data_predict = sc.inverse_transform(data_predict)
dataY_plot = sc.inverse_transform(dataY_plot)

plt.axvline(x=train_size, c='g', linestyle='--')

plt.plot(dataY_plot,'b',label='actual')
plt.plot(data_predict,'r',label='prediction')
plt.suptitle('Time Series Prediction')
plt.legend()
plt.show()

## Activity: Time Series Forecasting (Shampoo Sales)

#### Step 1: Preprocess Data (Shampoo Dataset)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

In [ ]:
training_set = pd.read_csv('shampoo.csv')
training_set.head(10)

In [ ]:
training_set.describe()

In [ ]:
training_set = training_set.iloc[:,1:2].values

plt.plot(training_set, label = 'Data')
plt.show()

In [ ]:
def sliding_window(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

In [ ]:
sc = MinMaxScaler()
training_data = sc.fit_transform(training_set)

seq_length = 3
x, y = sliding_window(training_data, seq_length)

In [ ]:
train_size = int(len(y) * 0.67)
test_size = int(len(y)) - train_size

dataX = torch.Tensor(np.array(x))
dataY = torch.Tensor(np.array(y))

trainX = torch.Tensor(np.array(x[0:train_size]))
trainY = torch.Tensor(np.array(y[0:train_size]))

testX = torch.Tensor(np.array(x[train_size:len(x)]))
testY = torch.Tensor(np.array(y[train_size:len(y)]))

#### Step 2: Define the Model

In [ ]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # Propagate input through LSTM
        _, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size)
        out = self.fc(h_out)
        
        return out

In [ ]:
input_size = 1
hidden_size = 3
num_layers = 1
num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

#### Step 3: Define Loss Function and Optimizer

In [ ]:
learning_rate = 0.01

criterion = torch.nn.MSELoss()    
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

#### Step 4: Train the Model

In [ ]:
num_epochs = 2000

for epoch in range(num_epochs):
    outputs = lstm(trainX)
    
    optimizer.zero_grad()
    loss = criterion(outputs, trainY)
    loss.backward()
    optimizer.step()

    
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

#### Step 5: Evalaute the Model

In [ ]:
lstm.eval()
train_predict = lstm(dataX)

data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

data_predict = sc.inverse_transform(data_predict)
dataY_plot = sc.inverse_transform(dataY_plot)

plt.axvline(x=train_size, c='g', linestyle='--')

plt.plot(dataY_plot,'b',label='actual')
plt.plot(data_predict,'r',label='prediction')
plt.suptitle('Time Series Prediction')
plt.legend()
plt.show()

## Activity: Stock Price Prediction

#### Step 1: Preporcess the data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

In [ ]:
training_set = pd.read_csv('AAPL.csv',usecols=['Date','Close'])
training_set.head(10)

In [ ]:
training_set.describe()

In [ ]:
training_set = training_set.iloc[:,1:2].values

plt.plot(training_set, label = 'Data')
plt.xlabel('Month')
plt.ylabel('Stock price')
plt.show()

In [ ]:
def sliding_window(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

In [ ]:
sc = MinMaxScaler()
training_data = sc.fit_transform(training_set)

seq_length = 4
x, y = sliding_window(training_data, seq_length)

In [ ]:
train_size = int(len(y) * 0.67)
test_size = int(len(y)) - train_size

dataX = torch.Tensor(np.array(x))
dataY = torch.Tensor(np.array(y))

trainX = torch.Tensor(np.array(x[0:train_size]))
trainY = torch.Tensor(np.array(y[0:train_size]))

testX = torch.Tensor(np.array(x[train_size:len(x)]))
testY = torch.Tensor(np.array(y[train_size:len(y)]))

#### Step 2: Define the Model

In [ ]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size)
        out = self.fc(h_out)
        
        return out

In [ ]:
input_size = 1
hidden_size = 2
num_layers = 1
num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

#### Step 3: Define Loss Function and Optimizer

In [ ]:
learning_rate = 0.01

criterion = torch.nn.MSELoss()    
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

#### Step 4: Train the Model

In [ ]:
num_epochs = 2000

for epoch in range(num_epochs):
    outputs = lstm(trainX)
    
    optimizer.zero_grad()
    loss = criterion(outputs, trainY)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

#### Step 5: Train the Model

In [ ]:
train_predict = lstm(dataX)

data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

data_predict = sc.inverse_transform(data_predict)
dataY_plot = sc.inverse_transform(dataY_plot)

plt.axvline(x=train_size, c='g', linestyle='--')

plt.plot(dataY_plot,'b',label='actual')
plt.plot(data_predict,'r',label='prediction')
plt.suptitle('Time Series Prediction for stock price')
plt.xlabel('Month')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

# Topic 2 Natural Language Processing

## Word Vectorization

### Bag-of-Words Representation (BoW)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside in a hot day.'
    ]
    
vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
vectorizer.transform(['My dogs like hot dogs on a hot hot day day day day.']).toarray()


### Activity: Bag-of-Words Model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters.',
        'Oil and Economy Cloud Stocks Outlook (Reuters) Reuters',
        'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters.',
        'Stocks End Up, But Near Year Lows (Reuters) Reuter',
    ]
vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
vectorizer.transform(['Oil price has gone up near new year from (Reuters) Reuters.']).toarray()

### N-gram BoW Model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))

corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside in a hot day.'
    ]

vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
vectorizer.transform(['My dogs like hot dogs on a hot hot day day day day.']).toarray()

### Activity:  N-gram BoW Model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))

corpus = [
        'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters.',
        'Oil and Economy Cloud Stocks Outlook (Reuters) Reuters',
        'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters.',
        'Stocks End Up, But Near Year Lows (Reuters) Reuter',
    ]
    
vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
vectorizer.transform(['Oil price has gone up near new year from (Reuters) Reuters.']).toarray()

### TF-IDF Vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside in a hot day.'
    ]

vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
vectorizer.transform(['My dogs like hot dogs on a hot hot day day day day.']).toarray()


### Activity: TF-IDF Vectorizaton

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))

corpus = [
        'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters.',
        'Oil and Economy Cloud Stocks Outlook (Reuters) Reuters',
        'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters.',
        'Stocks End Up, But Near Year Lows (Reuters) Reuter',
    ]

vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
vectorizer.transform(['Oil price has gone up near new year from (Reuters) Reuters.']).toarray()

## Word Embedding

In [ ]:
# !pip install -r requirement.txt
!pip install -r https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/nlp-pytorch/requirements.txt

### Train Embedding Layer

In [ ]:
import torch
import torchtext
import os
import collections

### Load TorchText Dataset
The torchtext library provides a few raw dataset iterators, which yield the raw text strings. For example, the ``AG_NEWS`` dataset iterators yield the raw data as a tuple of label and text.

In [ ]:
from torchtext.datasets import AG_NEWS
train_dataset, test_dataset = AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

In [ ]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

In [ ]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

The first step is to build a vocabulary with the raw training dataset.

In [ ]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.Vocab(counter, min_freq=1)

In [ ]:
vocab_size = len(vocab)
print(f"Vocab size is {vocab_size}")

In [ ]:
def encode(x):
    return [vocab.stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

By using embedding layer as a first layer in our network, we can switch from bag-or-words to embedding bag model, where we first convert each word in our text into corresponding embedding, and then compute some aggregate function over all those embeddings, such as sum, average or max

In [ ]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

As a result of this architecture, minibatches to our network would need to be created in a certain way. In the previous unit, when using bag-of-words, all BoW tensors in a minibatch had equal size vocab_size, regardless of the actual length of our text sequence. Once we move to word embeddings, we would end up with variable number of words in each text sample, and when combining those samples into minibatches we would have to apply some padding.

This can be done using the same technique of providing collate_fn function to the datasource:

In [ ]:
def padify(b):
    v = [encode(x[1]) for x in b]
    l = max(map(len,v))
    return ( 
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

In [ ]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

Note: We are only training for 25k records here (less than one full epoch) for the sake of time, but you can continue training, write a function to train for several epochs, and experiment with learning rate parameter to achieve higher accuracy. You should be able to go to the accuracy of about 90%.

In [ ]:
train_epoch(net,train_loader, lr=1, epoch_size=25000)

### Word2Vec

In [ ]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [ ]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

In [ ]:
w2v.word_vec('play')[:20]

In [ ]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

## Text Classification

In [ ]:
import torch
import torchtext
import os
import collections

we will start with a simple text classification task based on AG_NEWS dataset, which is to classify news headlines into one of 4 categories: World, Sports, Business and Sci/Tech. This dataset is built into torchtext module, so we can easily access it

In [ ]:
os.makedirs('./data',exist_ok=True)

In [ ]:
from torchtext.datasets import AG_NEWS
train_dataset, test_dataset = AG_NEWS(root='./data')

classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Here, train_dataset and test_dataset contain iterators that return pairs of label (number of class) and text respectively, for example

In [ ]:
next(train_dataset)

So, let's print out the first 10 new headlines from our dataset:

In [ ]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")

Because datasets are iterators, if we want to use the data multiple times we need to convert it to list:

In [ ]:
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

#### Tokenizer and Vocabulary

Now we need to convert text into numbers that can be represented as tensors. If we want word-level representation, we need to do two things:

- use tokenizer to split text into tokens
- build a vocabulary of those tokens.

In [ ]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

In [ ]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.Vocab(counter, min_freq=1)

In [ ]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

In [ ]:
def encode(x):
    return [vocab.stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

#### Create Custom BoW in Pytorch

In [ ]:
def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

In [ ]:
from torch.utils.data import DataLoader
import numpy as np 

def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

In [ ]:
def padify(b):
    v = [encode(x[1]) for x in b]
    l = max(map(len,v))
    return ( 
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

#### RNN Model

In [ ]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

RNNs are quite difficult to train, because once the RNN cells are unrolled along the sequence length, the resulting number of layers involved in back propagation is quite large. Thus we need to select small learning rate, and train the network on larger dataset to produce good results. It can take quite a long time, so using GPU is preferred.

In [ ]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    
    net.train()
    total_loss,acc,count,i = 0,0,0,0

    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()

        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    
    return total_loss.item()/count, acc.item()/count

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = RNNClassifier(vocab_size,64,32,len(classes))
train_epoch(net,train_loader, lr=0.001)

## Activity: Text Classification

In [ ]:
import torch
import torchtext
import os
import collections

In [ ]:
os.makedirs('./data',exist_ok=True)

In [ ]:
from torchtext.datasets import AG_NEWS
train_dataset, test_dataset = AG_NEWS(root='./data')

classes = ['World', 'Sports', 'Business', 'Sci/Tech']

In [ ]:
next(train_dataset)

In [ ]:
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

In [ ]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

In [ ]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.Vocab(counter)

In [ ]:
vocab_size = len(vocab)
print(f"Vocab size is {vocab_size}")

In [ ]:
def encode(x):
    return [vocab.stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

In [ ]:
def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

In [ ]:
from torch.utils.data import DataLoader
import numpy as np 

def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

In [ ]:
def padify(b):
    v = [encode(x[1]) for x in b]
    l = max(map(len,v))
    return ( 
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

#### LSTM Model

In [ ]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(_______, _______)
        self.rnn = torch.nn.LSTM(_________,__________,batch_first=True)
        self.fc = torch.nn.Linear(________,____________)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

#### Train the Model

In [ ]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):

    optimizer = torch.optim.Adam(net.parameters(),lr=lr)
    
    net.train()
    total_loss,acc,count,i = 0,0,0,0

    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()

        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    
    return total_loss.item()/count, acc.item()/count

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = LSTMClassifier(vocab_size,64,32,len(classes))
train_epoch(net,train_loader, lr=0.001)

## Solution: Text Classification

#### Import Dataaset

In [ ]:
import torch
import torchtext
import os
import collections

In [ ]:
os.makedirs('./data',exist_ok=True)

In [ ]:
from torchtext.datasets import AG_NEWS
train_dataset, test_dataset = AG_NEWS(root='./data')

classes = ['World', 'Sports', 'Business', 'Sci/Tech']

In [ ]:
next(train_dataset)

In [ ]:
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

In [ ]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

In [ ]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.Vocab(counter)

In [ ]:
vocab_size = len(vocab)
print(f"Vocab size is {vocab_size}")

In [ ]:
def encode(x):
    return [vocab.stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

In [ ]:
def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

In [ ]:
from torch.utils.data import DataLoader
import numpy as np 

def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

In [ ]:
def padify(b):
    v = [encode(x[1]) for x in b]
    l = max(map(len,v))
    return ( 
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

#### LSTM Model

In [ ]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        # self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

#### Train the Model

In [ ]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):

    optimizer = torch.optim.Adam(net.parameters(),lr=lr)
    
    net.train()
    total_loss,acc,count,i = 0,0,0,0

    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()

        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    
    return total_loss.item()/count, acc.item()/count

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = LSTMClassifier(vocab_size,64,32,len(classes))
train_epoch(net,train_loader, lr=0.001)

## Activity: Text Classificaiton 2

### Step 1: Import IMDB Dataset

In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

TEXT = data.Field(tokenize = 'spacy',tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

In [ ]:
print(vars(train_data.examples[0]))

In [ ]:
import random
SEED = 1

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

In [ ]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

In [ ]:
print(LABEL.vocab.stoi)

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainloader, validloader, testloader = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

### Step 2: Define the Model

In [ ]:
import torch.nn as nn

class GRU(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):       
        super().__init__()
        self.embedding = nn.Embedding(_____, __________)    
        self.gru = nn.GRU(_____dim, ____)
        self.fc = nn.Linear(_____, ______)
        
    def forward(self, text):
        embedded = self.embedding(text)    
        output, h_out = self.gru(embedded)       
        assert torch.equal(output[-1,:,:], h_out.squeeze(0))
        return self.fc(h_out.squeeze(0))

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = GRU(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = model.to(device)

### Step 3: Loss and Optimizer

In [ ]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

### Step 4 Train the Model

In [ ]:
def train():
    
    epoch_loss = 0

    model.train()
    for (X,y) in trainloader:
        optimizer.zero_grad()
        yhat = model(X).squeeze(1)
        loss = criterion(yhat, y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(trainloader)

In [ ]:
def evaluate():
    
    epoch_loss = 0
    epoch_acc = 0

    model.eval()
    with torch.no_grad():
        for (X,y) in validloader:
            yhat = model(X).squeeze(1)
            loss = criterion(yhat, y)

            epoch_loss += loss.item()

            preds = torch.round(torch.sigmoid(yhat))
            correct = (preds == y)
            acc = correct.sum() / len(y)
            epoch_acc += acc.item()
        
    return epoch_loss / len(validloader), epoch_acc / len(validloader)

In [ ]:
def test():
    
    epoch_loss = 0
    epoch_acc = 0

    model.eval()
    with torch.no_grad():
        for (X,y) in testloader:
            yhat = model(X).squeeze(1)
            loss = criterion(yhat, y)

            epoch_loss += loss.item()

            preds = torch.round(torch.sigmoid(yhat))
            correct = (preds == y)
            acc = correct.sum() / len(y)
            epoch_acc += acc.item()
        
    return epoch_loss / len(testloader), epoch_acc / len(testloader)

In [ ]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    train_loss = train()
    valid_loss, valid_acc = evaluate()

    print(f'Epoch: {epoch+1} Train Loss: {train_loss:.3f} | Val Loss: {valid_loss:.3f} |  Val Acc: {valid_acc*100:.2f}%')

### Step 5 Test the Model

In [ ]:
test_loss, test_acc = test()
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

## Solution : Text Classificaiton 2

### Step 1: Import IMDB Dataset

In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

TEXT = data.Field(tokenize = 'spacy',tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

In [ ]:
print(vars(train_data.examples[0]))

In [ ]:
import random
SEED = 1
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

In [ ]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

In [ ]:
print(LABEL.vocab.stoi)

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainloader, validloader, testloader = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

### Step 2: Define the Model

In [ ]:
import torch.nn as nn

class GRU(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):       
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)    
        self.gru = nn.GRU(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text)    
        output, h_out = self.gru(embedded)       
        assert torch.equal(output[-1,:,:], h_out.squeeze(0))
        return self.fc(h_out.squeeze(0))

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = GRU(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = model.to(device)

### Step 3: Loss and Optimizer

In [ ]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

### Step 4 Train the Model

In [ ]:
def train():
    
    epoch_loss = 0

    model.train()
    for (X,y) in trainloader:
        optimizer.zero_grad()
        yhat = model(X).squeeze(1)
        loss = criterion(yhat, y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(trainloader)

In [ ]:
def evaluate():
    
    epoch_loss = 0
    epoch_acc = 0

    model.eval()
    with torch.no_grad():
        for (X,y) in validloader:
            yhat = model(X).squeeze(1)
            loss = criterion(yhat, y)

            epoch_loss += loss.item()

            preds = torch.round(torch.sigmoid(yhat))
            correct = (preds == y)
            acc = correct.sum() / len(y)
            epoch_acc += acc.item()
        
    return epoch_loss / len(validloader), epoch_acc / len(validloader)

In [ ]:
def test():
    
    epoch_loss = 0
    epoch_acc = 0

    model.eval()
    with torch.no_grad():
        for (X,y) in testloader:
            yhat = model(X).squeeze(1)
            loss = criterion(yhat, y)

            epoch_loss += loss.item()

            preds = torch.round(torch.sigmoid(yhat))
            correct = (preds == y)
            acc = correct.sum() / len(y)
            epoch_acc += acc.item()
        
    return epoch_loss / len(testloader), epoch_acc / len(testloader)

In [ ]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    train_loss = train()
    valid_loss, valid_acc = evaluate()

    print(f'Epoch: {epoch+1} Train Loss: {train_loss:.3f} | Val Loss: {valid_loss:.3f} |  Val Acc: {valid_acc*100:.2f}%')

### Step 5 Test the Model

In [ ]:
test_loss, test_acc = test()
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

# Topic 3 Introduction to Attention Mechanism and Transformer Network

In [ ]:
!pip install transformers datasets

## Sequence Classification
Sequence classification is the task of classifying sequences according to a given number of classes. An example of sequence classification is the GLUE dataset, which is entirely based on that task

In [ ]:
import torch
import torchtext
import transformers
from transformers import pipeline

### Sentimental Analysis

In [ ]:
classifier = pipeline("sentiment-analysis")

In [ ]:
result = classifier("I hate you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

In [ ]:
result = classifier("I love you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

### Paraphase Detection

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ["not paraphrase", "is paraphrase"]

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

# The tokenizer will automatically add any model specific separators (i.e. <CLS> and <SEP>) and tokens to
# the sequence, as well as compute the attention masks.
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="pt")

paraphrase_classification_logits = model(**paraphrase).logits
not_paraphrase_classification_logits = model(**not_paraphrase).logits

paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]

# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")

# Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")



## Extractive Question Answerin
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a question answering dataset is the SQuAD dataset, which is entirely based on that task.

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")

context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.
"""

This returns an answer extracted from the text, a confidence score, alongside "start" and "end" values, which are the
positions of the extracted answer in the text

In [ ]:
result = question_answerer(question="What is extractive question answering?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
result = question_answerer(question="What is a good example of a question answering dataset?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Here is an example of question answering using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a BERT model and loads it
   with the weights stored in the checkpoint.
2. Define a text and a few questions.
3. Iterate over the questions and build a sequence from the text and the current question, with the correct
   model-specific separators token type ids and attention masks.
4. Pass this sequence through the model. This outputs a range of scores across the entire sequence tokens (question and
   text), for both the start and end positions.
5. Compute the softmax of the result to get probabilities over the tokens.
6. Fetch the tokens from the identified start and stop values, convert those tokens to a string.
7. Print the results.

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits
    # Get the most likely beginning of answer with the argmax of the score
    answer_start = torch.argmax(answer_start_scores)
    # Get the most likely end of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    print(f"Question: {question}")
    print(f"Answer: {answer}")

## Language Modeling
Language modeling is the task of fitting a model to a corpus, which can be domain specific. All popular
transformer-based models are trained using a variant of language modeling, e.g. BERT with masked language modeling,
GPT-2 with causal language modeling.

Language modeling can be useful outside of pretraining as well, for example to shift the model distribution to be
domain-specific: using a language model trained over a very large corpus, and then fine-tuning it to a news dataset or
on scientific papers

### Masked Language Modeling
Masked language modeling is the task of masking tokens in a sequence with a masking token, and prompting the model to fill that mask with an appropriate token. This allows the model to attend to both the right context (tokens on the right of the mask) and the left context (tokens on the left of the mask). Such a training creates a strong basis for downstream tasks requiring bi-directional context, such as SQuAD (question answering


In [ ]:
from transformers import pipeline

unmasker = pipeline("fill-mask")

This outputs the sequences with the mask filled, the confidence score, and the token id in the tokenizer vocabulary:

In [ ]:
from pprint import pprint
pprint(unmasker(f"HuggingFace is creating a {unmasker.tokenizer.mask_token} that the community uses to solve NLP tasks."))

Here is an example of doing masked language modeling using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a DistilBERT model and
   loads it with the weights stored in the checkpoint.
2. Define a sequence with a masked token, placing the `tokenizer.mask_token` instead of a word.
3. Encode that sequence into a list of IDs and find the position of the masked token in that list.
4. Retrieve the predictions at the index of the mask token: this tensor has the same size as the vocabulary, and the
   values are the scores attributed to each token. The model gives higher score to tokens it deems probable in that
   context.
5. Retrieve the top 5 tokens using the PyTorch `topk` or TensorFlow `top_k` methods.
6. Replace the mask token by the tokens and print the results

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-cased")

sequence = "Distilled models are smaller than the models they mimic. Using them instead of the large " \
    f"versions would help {tokenizer.mask_token} our carbon footprint."

inputs = tokenizer(sequence, return_tensors="pt")
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

token_logits = model(**inputs).logits
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

This prints five sequences, with the top 5 tokens predicted by the model:

In [ ]:
for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

### Causal Language Modeling
Causal language modeling is the task of predicting the token following a sequence of tokens. In this situation, the
model only attends to the left context (tokens on the left of the mask). Such a training is particularly interesting
for generation tasks. 

Usually, the next token is predicted by sampling from the logits of the last hidden state the model produces from the
input sequence.

Here is an example of using the tokenizer and model and leveraging the
`PreTrainedModel.top_k_top_p_filtering` method to sample the next token following an input sequence
of tokens.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch import nn

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

sequence = f"Hugging Face is based in DUMBO, New York City, and"

inputs = tokenizer(sequence, return_tensors="pt")
input_ids = inputs["input_ids"]

# get logits of last hidden state
next_token_logits = model(**inputs).logits[:, -1, :]

# filter
filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)

# sample
probs = nn.functional.softmax(filtered_next_token_logits, dim=-1)
next_token = torch.multinomial(probs, num_samples=1)

generated = torch.cat([input_ids, next_token], dim=-1)

resulting_string = tokenizer.decode(generated.tolist()[0])

This outputs a (hopefully) coherent next token following the original sequence, which in our case is the word **has**:

In [ ]:
print(resulting_string)

## Text Generation
In text generation (a.k.a open-ended text generation) the goal is to create a coherent portion of text that is a continuation from the given context. The following example shows how GPT-2 can be used in pipelines to generate text.

In [ ]:
from transformers import pipeline

text_generator = pipeline("text-generation")
print(text_generator("As far as I am concerned, I will", max_length=50, do_sample=False))

Here, the model generates a random text with a total maximal length of **50** tokens from context **"As far as I am
concerned, I will"**. Behind the scenes, the pipeline object calls the method
`PreTrainedModel.generate` to generate text. The default arguments for this method can be
overridden in the pipeline, as is shown above for the arguments `max_length` and `do_sample`.

Below is an example of text generation using `XLNet` and its tokenizer, which includes calling `generate` directly:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("xlnet-base-cased")
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology
PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""

prompt = "Today the weather is really nice and I am planning on "
inputs = tokenizer(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]

prompt_length = len(tokenizer.decode(inputs[0]))
outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = prompt + tokenizer.decode(outputs[0])[prompt_length+1:]

In [ ]:
print(generated)

## Named Entity Recognition
Named Entity Recognition (NER) is the task of classifying tokens according to a class, for example, identifying a token as a person, an organisation or a location. An example of a named entity recognition dataset is the CoNLL-2003 dataset, which is entirely based on that task.

Here is an example of using pipelines to do named entity recognition, specifically, trying to identify tokens as belonging to one of 9 classes:

- O, Outside of a named entity
- B-MIS, Beginning of a miscellaneous entity right after another miscellaneous entity
- I-MIS, Miscellaneous entity
- B-PER, Beginning of a person's name right after another person's name
- I-PER, Person's name
- B-ORG, Beginning of an organisation right after another organisation
- I-ORG, Organisation
- B-LOC, Beginning of a location right after another location
- I-LOC, Location

In [ ]:
from transformers import pipeline

ner_pipe = pipeline("ner")

sequence = """Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO,
therefore very close to the Manhattan Bridge which is visible from the window."""

In [ ]:
for entity in ner_pipe(sequence):
    print(entity)

Note how the tokens of the sequence "Hugging Face" have been identified as an organisation, and "New York City",
"DUMBO" and "Manhattan Bridge" have been identified as locations.

Here is an example of doing named entity recognition, using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a BERT model and loads it
   with the weights stored in the checkpoint.
2. Define a sequence with known entities, such as "Hugging Face" as an organisation and "New York City" as a location.
3. Split words into tokens so that they can be mapped to predictions. We use a small hack by, first, completely
   encoding and decoding the sequence, so that we're left with a string that contains the special tokens.
4. Encode that sequence into IDs (special tokens are added automatically).
5. Retrieve the predictions by passing the input to the model and getting the first output. This results in a
   distribution over the 9 possible classes for each token. We take the argmax to retrieve the most likely class for
   each token.
6. Zip together each token with its prediction and print it.

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, " \
           "therefore very close to the Manhattan Bridge."

inputs = tokenizer(sequence, return_tensors="pt")
tokens = inputs.tokens()

outputs = model(**inputs).logits
predictions = torch.argmax(outputs, dim=2)

This outputs a list of each token mapped to its corresponding prediction. Differently from the pipeline, here every
token has a prediction as we didn't remove the "0"th class, which means that no particular entity was found on that
token.

In the above example, `predictions` is an integer that corresponds to the predicted class. We can use the
`model.config.id2label` property in order to recover the class name corresponding to the class number, which is
illustrated below:

In [ ]:
for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

## Summarization
Summarization is the task of summarizing a document or an article into a shorter text. 

An example of a summarization dataset is the CNN / Daily Mail dataset, which consists of long news articles and was
created for the task of summarization. 

Here is an example of using the pipelines to do summarization. It leverages a Bart model that was fine-tuned on the CNN
/ Daily Mail data set.

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

Because the summarization pipeline depends on the PreTrainedModel.generate() method, we can override the default arguments of PreTrainedModel.generate() directly in the pipeline for max_length and min_length as shown below. This outputs the following summary:

In [ ]:
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer("summarize: " + ARTICLE, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0]))

## Translation
Translation is the task of translating a text from one language to another. 

An example of a translation dataset is the WMT English to German dataset, which has sentences in English as the input
data and the corresponding sentences in German as the target data. 

Here is an example of using the pipelines to do translation. It leverages a T5 model that was only pre-trained on a
multi-task mixture dataset (including WMT), yet, yielding impressive translation results.

In [ ]:
from transformers import pipeline

translator = pipeline("translation_en_to_de")
print(translator("Hugging Face is a technology company based in New York and Paris", max_length=40))

Because the translation pipeline depends on the `PreTrainedModel.generate()` method, we can override the default
arguments of `PreTrainedModel.generate()` directly in the pipeline as is shown for `max_length` above.

Here is an example of doing translation using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. Summarization is usually done using an encoder-decoder
   model, such as `Bart` or `T5`.
2. Define the article that should be summarized.
3. Add the T5 specific prefix "translate English to German: "
4. Use the `PreTrainedModel.generate()` method to perform the translation.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

inputs = tokenizer(
    "translate English to German: Hugging Face is a technology company based in New York and Paris",
    return_tensors="pt"
)
outputs = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)

print(tokenizer.decode(outputs[0]))

As with the pipeline example, we get the same translation:

In [ ]:
print(tokenizer.decode(outputs[0]))

# Appendix 1: Datasets

In [ ]:
from torchtext.datasets import AG_NEWS
train_iter = AG_NEWS(split='train')
next(train_iter)

# from torchtext.datasets import AG_NEWS
# train_dataset, test_dataset = AG_NEWS(root='./data')
# next(train_dataset)


In [ ]:
# Or iterate with for loop
for (label, line) in train_iter:
    print(label, line)

In [ ]:
# Or send to DataLoader
from torch.utils.data import DataLoader

train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False)

In [ ]:
from torchtext.datasets import IMDA
train_iter = IMDA(split='train')
next(train_iter)

# Appendix 2: Using Pre-Trained Embeddings in PyTorch

In [ ]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.itos):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

In [ ]:
def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [ ]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)